# মেটা ফ্যামিলি মডেল নিয়ে কাজ করা

## পরিচিতি

এই পাঠে আলোচনা করা হবে:

- মেটা ফ্যামিলির দুটি প্রধান মডেল - Llama 3.1 এবং Llama 3.2 নিয়ে জানবো
- প্রতিটি মডেলের ব্যবহার ক্ষেত্র ও উপযোগিতা বোঝা
- প্রতিটি মডেলের বিশেষ বৈশিষ্ট্য দেখানোর জন্য কোডের নমুনা

## মেটা ফ্যামিলির মডেলসমূহ

এই পাঠে আমরা মেটা ফ্যামিলি বা "Llama Herd" থেকে ২টি মডেল - Llama 3.1 এবং Llama 3.2 নিয়ে আলোচনা করবো

এই মডেলগুলো বিভিন্ন ভ্যারিয়েন্টে আসে এবং Github Model মার্কেটপ্লেসে পাওয়া যায়। Github Models ব্যবহার করে [AI মডেল দিয়ে প্রোটোটাইপ তৈরি](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) করার বিষয়ে আরও জানতে এখানে দেখুন।

মডেল ভ্যারিয়েন্টসমূহ:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*নোট: Llama 3-ও Github Models-এ পাওয়া যায়, তবে এই পাঠে তা আলোচনা করা হবে না*


## Llama 3.1

৪০৫ বিলিয়ন প্যারামিটারে, Llama 3.1 ওপেন সোর্স LLM ক্যাটাগরিতে পড়ে।

এই মডেলটি আগের ভার্সন Llama 3 থেকে আপগ্রেড, যা দেয়:

- বড় কনটেক্সট উইন্ডো - ১২৮কে টোকেন বনাম ৮কে টোকেন
- বড় ম্যাক্স আউটপুট টোকেন - ৪০৯৬ বনাম ২০৪৮
- আরও ভালো বহু-ভাষার সাপোর্ট - ট্রেনিং টোকেন বাড়ার কারণে

এসবের ফলে Llama 3.1 আরও জটিল ব্যবহারিক ক্ষেত্র সামলাতে পারে, যেমন GenAI অ্যাপ্লিকেশন তৈরিতে:

- নেটিভ ফাংশন কলিং - LLM ওয়ার্কফ্লোর বাইরে এক্সটার্নাল টুল ও ফাংশন কল করার ক্ষমতা
- আরও ভালো RAG পারফরম্যান্স - বড় কনটেক্সট উইন্ডোর কারণে
- সিনথেটিক ডেটা জেনারেশন - ফাইন-টিউনিংয়ের মতো কাজে কার্যকর ডেটা তৈরি করার ক্ষমতা


### নেটিভ ফাংশন কলিং

Llama 3.1-কে আরও কার্যকরভাবে ফাংশন বা টুল কল করার জন্য ফাইন-টিউন করা হয়েছে। এতে দুটি বিল্ট-ইন টুল রয়েছে, যেগুলো মডেল ব্যবহারকারীর প্রম্পট অনুযায়ী ব্যবহার করা দরকার কিনা তা নিজে চিনতে পারে। এই টুলগুলো হলো:

- **Brave Search** - ওয়েব সার্চের মাধ্যমে আবহাওয়ার মতো আপডেটেড তথ্য পেতে ব্যবহার করা যায়
- **Wolfram Alpha** - আরও জটিল গাণিতিক হিসাবের জন্য ব্যবহার করা যায়, ফলে নিজে ফাংশন লেখার দরকার হয় না।

আপনি চাইলে নিজের কাস্টম টুলও তৈরি করতে পারেন, যেগুলো LLM কল করতে পারবে।

নিচের কোড উদাহরণে:

- আমরা সিস্টেম প্রম্পটে (brave_search, wolfram_alpha) টুলগুলো ডিফাইন করি।
- ব্যবহারকারীর প্রম্পট পাঠানো হয়, যেখানে নির্দিষ্ট একটি শহরের আবহাওয়া জানতে চাওয়া হয়েছে।
- LLM Brave Search টুলে একটি টুল কল করবে, যা এভাবে দেখাবে `<|python_tag|>brave_search.call(query="Stockholm weather")`

*নোট: এই উদাহরণে শুধু টুল কল করা হয়েছে, যদি আপনি ফলাফল পেতে চান, তাহলে আপনাকে Brave API পেজে একটি ফ্রি অ্যাকাউন্ট খুলতে হবে এবং ফাংশনটি নিজে ডিফাইন করতে হবে*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

LLM হলেও, Llama 3.1-এর একটি সীমাবদ্ধতা হলো মাল্টিমোডালিটি। অর্থাৎ, বিভিন্ন ধরনের ইনপুট যেমন ছবি ব্যবহার করে প্রম্পট দেওয়া এবং তার উত্তর দেওয়া। এই ক্ষমতাই Llama 3.2-এর অন্যতম প্রধান বৈশিষ্ট্য। এই ফিচারগুলোর মধ্যে রয়েছে:

- মাল্টিমোডালিটি - টেক্সট এবং ছবি, দুই ধরনের প্রম্পট বিশ্লেষণ করতে পারে
- ছোট থেকে মাঝারি সাইজের ভ্যারিয়েশন (11B এবং 90B) - এতে ডিপ্লয়মেন্টের জন্য নমনীয়তা পাওয়া যায়,
- শুধু টেক্সট-ভিত্তিক ভ্যারিয়েশন (1B এবং 3B) - এতে মডেলটি এজ বা মোবাইল ডিভাইসে ডিপ্লয় করা যায় এবং কম লেটেন্সি পাওয়া যায়

মাল্টিমোডাল সাপোর্ট ওপেন সোর্স মডেলের জগতে একটি বড় অগ্রগতি। নিচের কোড উদাহরণে একটি ছবি এবং টেক্সট প্রম্পট একসাথে ব্যবহার করে Llama 3.2 90B থেকে ছবিটির বিশ্লেষণ পাওয়া যায়।

### Llama 3.2-এ মাল্টিমোডাল সাপোর্ট


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## শেখা এখানেই শেষ নয়, যাত্রা চালিয়ে যান

এই পাঠটি শেষ করার পর, আমাদের [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) দেখুন, যাতে আপনি আপনার Generative AI জ্ঞানের স্তর আরও বাড়াতে পারেন!



---

**দায়িত্ব অস্বীকার**:
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব নির্ভুল অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন, স্বয়ংক্রিয় অনুবাদে ভুল বা অসঙ্গতি থাকতে পারে। মূল ভাষায় রচিত নথিটিকেই চূড়ান্ত ও নির্ভরযোগ্য উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ গ্রহণ করার পরামর্শ দেওয়া হচ্ছে। এই অনুবাদের ব্যবহারে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
